In [ ]:
#@title Show NODEID

import os
import hashlib

#print(os.environ["HOSTNAME"])
NODEID = hashlib.md5(os.environ["HOSTNAME"].encode('utf-8')).hexdigest().upper()
print("NODEID: " + NODEID)

# INIT

In [ ]:
#@title Clone CSRBnotebook
!git -C /content/ clone --recurse-submodules git://github.com/CSRBapp/CSRBnotebook.git

In [ ]:
#@title Update CSRBnotebook
!git -C /content/CSRBnotebook pull --recurse-submodules

# RUN


In [ ]:
#@title Start CSRBvfsFUSE

import os
import hashlib
import psutil

assert "CSRBvfsFUSE" not in (p.name() for p in psutil.process_iter()), ("CSRBvfsFUSE is already running")

#print(os.environ["HOSTNAME"])
NODEID = hashlib.md5(os.environ["HOSTNAME"].encode('utf-8')).hexdigest().upper()
print("NODEID: " + NODEID)

ROUTERHOST = "35.176.200.197" #@param {type: "string"}
ROUTERPORT = "32450" #@param {type: "string"}
ROUTERINTERSPACEUSEC = "200000" #@param {type: "string"}
CSRBNOTEBOOKROOT = "/content/CSRBnotebook/"
global CSRBVFS
CSRBVFS = "/mnt/CSRB" #@param {type: "string"}
RUNBG = True #@param {type: "boolean"}

print("Starting CSRBvfsFUSE " + ("in Background" if RUNBG is True else "in Foreground"))

!chmod a+x {CSRBNOTEBOOKROOT}/CSRBbin/UBUNTU-18.04/CSRBvfsFUSE
!mkdir {CSRBVFS} 2> /dev/null
!umount -f {CSRBVFS} 2> /dev/null

os.system(CSRBNOTEBOOKROOT + "/CSRBbin/UBUNTU-18.04/CSRBvfsFUSE -o " \
"nodev,allow_other,bindHost=0.0.0.0,bindPort=0," \
"routerHost=" + ROUTERHOST + ",routerPort=" + ROUTERPORT + "," \
"routerInterspaceUSEC=" + ROUTERINTERSPACEUSEC + "," \
"nodeCAcertificateFile=" + CSRBNOTEBOOKROOT + "/CSRBbin/CERTS/CA.nodes.csrb.crt," \
"nodeCertificateFile=" + CSRBNOTEBOOKROOT + "/CSRBbin/CERTS/C9BAD58F23D5A6C095C0571512CD318D.nodes.csrb.pem," \
"nodeID=" + NODEID + ",storagePath=/mnt/CSRBSTORAGE/" + NODEID + "/ " \
"-f /mnt/CSRB" + (" &" if RUNBG is True else ""))

In [ ]:
#@title Stop CSRBvfsFUSE

!pidof CSRBvfsFUSE > /dev/null && kill -9 `pidof CSRBvfsFUSE`
!umount -f {CSRBVFS} > /dev/null

# MISC

In [ ]:
!ls -l {CSRBVFS}

In [ ]:
!ps -awux | grep CSRBvfsFUSE

# CSRBappNotHotdog

## INIT

In [ ]:
#@title Clone CSRBappNotHostdogServer
!git -C /content/ clone --recurse-submodules https://github.com/CSRBapp/CSRBappNotHotdogServer.git

In [ ]:
#@title Update CSRBappNotHotdogServer
!git -C /content/CSRBappNotHotdogServer pull --recurse-submodules

In [ ]:
#@title Diff CSRBappNotHotdogServer
!git -C /content/CSRBappNotHotdogServer diff --submodule=diff

## RUN

In [ ]:
#@title Start CSRBappNotHotdog Server

os.chdir("/content/CSRBappNotHotdogServer/")
os.environ["CSRBVFS"] = CSRBVFS
%run CSRBappNotHotdog.py